In [86]:
import random
import numpy as np
from scipy.optimize import minimize

In [20]:
np.random.random((1,3))


array([[0.59220764, 0.19702821, 0.26924554]])

In [33]:
def generate_toy_data(N, num_views):
    d = [2, 3]
    y = np.random.randint(-1,2,size=[N,1])
    X = []

    # generate y
    for i in range(len(y)):
        if y[i][0] == 0:
            if random.random() < 0.5:
                y[i][0] = -1
            else:
                y[i][0] = 1
    print(y)
    
    for i in range(N):
        new_point = {}
        for j in range(num_views):
            new_point[j] = np.random.random((1,d[j]))
        X.append(new_point)
    print(X)
    return X, y

X, y = generate_toy_data(10, 2)

[[-1]
 [ 1]
 [ 1]
 [ 1]
 [-1]
 [-1]
 [-1]
 [ 1]
 [ 1]
 [ 1]]
[{0: array([[0.54677592, 0.77122567]]), 1: array([[0.44955297, 0.37366502, 0.29564167]])}, {0: array([[0.90246079, 0.9806609 ]]), 1: array([[0.93421342, 0.91938803, 0.6332184 ]])}, {0: array([[0.5831904 , 0.07170115]]), 1: array([[0.46603263, 0.20958875, 0.73081446]])}, {0: array([[0.47713206, 0.31765667]]), 1: array([[0.83274814, 0.91318138, 0.49768693]])}, {0: array([[0.55698515, 0.09418115]]), 1: array([[0.42498101, 0.60549777, 0.69846496]])}, {0: array([[0.92355087, 0.87704291]]), 1: array([[0.98008731, 0.84556952, 0.59344168]])}, {0: array([[0.51523902, 0.44489308]]), 1: array([[0.01129934, 0.98354954, 0.26929589]])}, {0: array([[0.55122197, 0.85590211]]), 1: array([[0.24067043, 0.16619771, 0.20575334]])}, {0: array([[0.30365086, 0.77560078]]), 1: array([[0.1804141 , 0.70662968, 0.39650927]])}, {0: array([[0.23884786, 0.79839158]]), 1: array([[0.87072032, 0.66618508, 0.74208244]])}]


In [58]:
X1 = np.zeros((10, 2))
X2 = np.zeros((10, 3))

for i in range(len(X)):
    X1[i] = X[i][0][0]
    X2[i] = X[i][1][0]
print(X1)
print(X2)

[[0.54677592 0.77122567]
 [0.90246079 0.9806609 ]
 [0.5831904  0.07170115]
 [0.47713206 0.31765667]
 [0.55698515 0.09418115]
 [0.92355087 0.87704291]
 [0.51523902 0.44489308]
 [0.55122197 0.85590211]
 [0.30365086 0.77560078]
 [0.23884786 0.79839158]]
[[0.44955297 0.37366502 0.29564167]
 [0.93421342 0.91938803 0.6332184 ]
 [0.46603263 0.20958875 0.73081446]
 [0.83274814 0.91318138 0.49768693]
 [0.42498101 0.60549777 0.69846496]
 [0.98008731 0.84556952 0.59344168]
 [0.01129934 0.98354954 0.26929589]
 [0.24067043 0.16619771 0.20575334]
 [0.1804141  0.70662968 0.39650927]
 [0.87072032 0.66618508 0.74208244]]


In [82]:
def k1(a, b, s, l):
    diff = a - b
    result = s * np.exp(np.inner(diff, diff) / (-2 * l))
#     print("result", result)
    return result

def k2(a, b, s, l):
    diff = a - b
    result = s * np.exp(np.inner(diff, diff) / (-2 * l))
#     print("result", result)
    return result

In [46]:
a = X[0][0][0]
b = X[1][0][0]
print(a, b)
np.inner(a, b)
k1(a, b, 1, 1)

[0.54677592 0.77122567] [0.90246079 0.9806609 ]
result 0.918340153175999


0.918340153175999

In [127]:
# the optimization variables
# [s1, l1, sigma1, s2, l2, sigma2]
N = 10
fixed_para = {'X1': X1, 'X2': X2, 'y': y}
x0 = [1, 1, 1, 1, 1, 1]
def obj_func(x_0, fixed_para):
    s1, l1, sigma1, s2, l2, sigma2 = x_0
    X1 = fixed_para['X1']
    X2 = fixed_para['X2']
    y = fixed_para['y']
    # construct K1
    K1 = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            K1[i][j] = k1(X1[i], X1[j], s1, l1)
#     print("K1: \n", K1, np.linalg.det(K1))

    
    K1_plus = K1 + sigma1 * np.eye(N)
    inv_K1_plus = np.linalg.inv(K1_plus)

    likelihood_1 = -0.5 * np.dot(np.dot(y.T, inv_K1_plus), y) - 0.5 * np.log(np.linalg.det(K1_plus)) - N/2*np.log(2*np.pi)

    
    # construct K2
    K2 = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            K2[i][j] = k2(X2[i], X2[j], s2, l2)
#     print("K2: \n", K2, np.linalg.det(K2))
    K2_plus = K2 + sigma2 * np.eye(N)
    inv_K2_plus = np.linalg.inv(K2_plus)

    likelihood_2 = -0.5 * np.dot(np.dot(y.T, inv_K2_plus), y) - 0.5 * np.log(np.linalg.det(K2_plus)) - N/2*np.log(2*np.pi)
    
    
    # construct mu_1, mu_2, Sigma_1, Sigma_2
    mu_1 = np.dot(np.dot(K1, inv_K1_plus), y)
    mu_2 = np.dot(np.dot(K2, inv_K2_plus), y)
    
    Sigma_1 = K1 - np.dot(np.dot(K1, inv_K1_plus), K1)
    Sigma_2 = K2 - np.dot(np.dot(K2, inv_K2_plus), K2)
    inv_Sigma_1 = np.linalg.inv(Sigma_1)
    inv_Sigma_2 = np.linalg.inv(Sigma_2)
    
    KL_12 = 0.5 * (np.log(np.linalg.det(Sigma_2)) - np.log(np.linalg.det(Sigma_1))\
                   + np.trace(np.dot(inv_Sigma_2, Sigma_1)) + np.dot(np.dot((mu_2 - mu_1).T, inv_Sigma_2), mu_2 - mu_1) - N)
    
    KL_21 = 0.5 * (np.log(np.linalg.det(Sigma_1)) - np.log(np.linalg.det(Sigma_2))\
                   + np.trace(np.dot(inv_Sigma_1, Sigma_2)) + np.dot(np.dot((mu_1 - mu_2).T, inv_Sigma_1), mu_1 - mu_2) - N)
    
    return -(likelihood_1 + likelihood_2) + KL_12 + KL_21


obj_func(x0, fixed_para)

array([[69647.89355688]])

In [133]:
minimize(obj_func, x0, fixed_para, method='nelder-mead', options={'xatol': 1e-8, 'disp': True, 'maxfevint':10000, 'maxiter':10000})

/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:1: OptimizeWarning: Unknown solver options: maxfevint
  """Entry point for launching an IPython kernel.
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in log
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in log
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in log
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in log
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/orres/anaconda3/envs/PAL/lib/python3.7/site-packages/ipy

Optimization terminated successfully.
         Current function value: -11462018768792656942029955033245676080177128899083905889456909320520365633811407926672852544625640231827884779573868968005533852177090478080.000000
         Iterations: 1087
         Function evaluations: 2024


 final_simplex: (array([[ 1.36054015e+00,  2.26945197e-04, -2.36647771e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647769e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647773e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647770e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647775e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647773e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02],
       [ 1.36054015e+00,  2.26945197e-04, -2.36647770e-16,
         2.78184825e+00,  4.85365298e-03,  2.41896099e-02]]), array([-1.14620188e+139, -1.14620188e+139, -1.14620188e+139,
       -1.14620188e+139, -1.14620188e+139, -1.14620188e+139,
       -1.14620188e+139]))